<a href="https://colab.research.google.com/github/vit050587/Natural_Language_Processing/blob/master/KVA_HW_12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#HW 12. Модель Transformer-2
Взять тот же датасет, который был на вебинаре и предобученную модель для задачи суммаризации
1. Проверить насколько хорошо она суммаризирует

In [2]:
!pip install transformers sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 40.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 76.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 102.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 80.0 MB/s eta 0:00:00


In [3]:
import json
import random
import pandas as pd
import re

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
def read_gazeta_records(file_name, shuffle=True, sort_by_date=False):
    assert shuffle != sort_by_date
    records = []
    with open(file_name, "r") as r:
        for line in r:
            records.append(json.loads(line))
    if sort_by_date:
        records.sort(key=lambda x: x["date"])
    if shuffle:
        random.shuffle
    return records

In [6]:
train_records = read_gazeta_records("/content/drive/MyDrive/Colab Notebooks/Natural_Language_Processing/lesson12/archive/gazeta_train.txt")
val_records = read_gazeta_records("/content/drive/MyDrive/Colab Notebooks/Natural_Language_Processing/lesson12/archive/gazeta_val.txt")
test_records = read_gazeta_records("/content/drive/MyDrive/Colab Notebooks/Natural_Language_Processing/lesson12/archive/gazeta_test.txt")

In [7]:
!pip install rouge
!pip install razdel

#Метод Lead-3 (экстрактивная суммаризация)

In [8]:
from nltk.translate.bleu_score import corpus_bleu
from rouge import Rouge

def calc_scores(references, predictions, metric="all"):
    print("Count:", len(predictions))
    print("Ref:", references[-1])
    print("Hyp:", predictions[-1])

    if metric in ("bleu", "all"):
        print("BLEU: ", corpus_bleu([[r] for r in references], predictions))
    if metric in ("rouge", "all"):
        rouge = Rouge()
        scores = rouge.get_scores(predictions, references, avg=True)
        print("ROUGE: ", scores)

In [9]:
import razdel

def calc_lead_n_score(records, summary_col = 'summary', n=3, lower=True, nrows=1000):
    references = []
    predictions = []

    for i, record in enumerate(records):
        if i >= nrows:
            break

        summary = record[summary_col]
        summary = summary if not lower else summary.lower()
        references.append(summary)

        text = record["text"]
        text = text if not lower else text.lower()
        sentences = [sentence.text for sentence in razdel.sentenize(text)]
        prediction = " ".join(sentences[:n])
        predictions.append(prediction)

    calc_scores(references, predictions)

calc_lead_n_score(test_records, n=1)

Count: 1000
Ref: в дтп с участием школьного автобуса, грузовиков и легковой «лада веста» на 339-м километре автодороги «кавказ» в кизлярском районе дагестана погибли три человека, еще девять получили травмы. находившиеся в автобусе дети ехали на тренировку из селения караузек в районный центр. республиканская прокуратура проверяет, проходил ли автобус предрейсовый контроль технического состояния.
Hyp: при столкновении школьного микроавтобуса с «газелью», двумя грузовиками «камаз», двумя фурами и легковой «лада веста» днем 7 октября в кизлярском районе дагестана на 339-м километре автодороги «кавказ» погибли три человека, еще девять получили травмы, сообщает пресс-служба республиканского мвд.
BLEU:  0.23525936446020249
ROUGE:  {'rouge-1': {'r': 0.3566712092461257, 'p': 0.23967100725490165, 'f': 0.2507416850786502}, 'rouge-2': {'r': 0.15156818123659455, 'p': 0.10249878626808365, 'f': 0.10567053444670313}, 'rouge-l': {'r': 0.32632964439324663, 'p': 0.21532288947552658, 'f': 0.226815223217

#Предобученный трансформер (абстрактивная суммаризация)

In [10]:
import torch

In [11]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
model_name = "csebuetnlp/mT5_multilingual_XLSum"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

You are using the legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:470: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [12]:
model.eval();

def summarize(
    text, n_words=None, compression=None,
    max_length=1000, num_beams=3, do_sample=False, repetition_penalty=10.0,
    **kwargs
):
    """
    Summarize the text
    The following parameters are mutually exclusive:
    - n_words (int) is an approximate number of words to generate.
    - compression (float) is an approximate length ratio of summary and original text.
    """
    if n_words:
        text = '[{}] '.format(n_words) + text
    elif compression:
        text = '[{0:.1g}] '.format(compression) + text
    x = tokenizer(text, return_tensors='pt', padding=True).to(model.device)
    with torch.inference_mode():
        out = model.generate(
            **x,
            max_length=max_length, num_beams=num_beams,
            do_sample=do_sample, repetition_penalty=repetition_penalty,
            **kwargs
        )
    return tokenizer.decode(out[0], skip_special_tokens=True)

In [13]:
df_test = pd.DataFrame(test_records)

In [14]:
df_test

,date,url,summary,title,text
0,2020-09-01 00:22:59,https://www.gazeta.ru/politics/2020/08/31_a_13...,Протестующие против антикоронавирусных мер нем...,В Германии объяснили упоминание имени Путина н...,На этих выходных в Берлине прошли крупные акци...
1,2020-09-01 08:08:16,https://www.gazeta.ru/politics/2020/08/31_a_13...,"Делегации Израиля и США прилетели в ОАЭ, где о...",Делегации Израиля и США прибыли в ОАЭ для обсу...,Высокопоставленная американская и израильская ...
2,2020-09-01 09:21:38,https://www.gazeta.ru/politics/2020/09/01_a_13...,Белорусская оппозиция в лице экс-кандидата в п...,Оппозиция Белоруссии объявила о создании новой...,Одна из руководителей Координационного совета ...
3,2020-09-01 09:33:30,https://www.gazeta.ru/army/2020/09/01/13222904...,Действия американских ВС в Эстонии во время уч...,Россия считает крайне опасными действия США на...,Россия считает действия ВС США во время учений...
4,2020-09-01 09:49:24,https://www.gazeta.ru/business/2020/09/01/1322...,Поправки в российский закон «О банкротстве» вс...,В России вступил в силу закон о внесудебном ба...,С 1 сентября в России вступают в силу поправки...
...,...,...,...,...,...
6788,2021-09-03 09:28:34,https://www.gazeta.ru/science/2021/09/03_a_139...,Первый пуск американской частной ракеты Alpha ...,СМИ назвали успехом запуск частной ракеты Alph...,Феерическим взрывом в атмосфере закончился пер...
6789,2021-09-03 09:45:36,https://www.gazeta.ru/economics/2021/09/03/139...,Городская агломерация Владивостока может дости...,На ВЭФ подписано соглашение о создании города ...,Глава Минвостокразвития Алексей Чекунков 31 ав...
6790,2021-09-03 10:44:03,https://www.gazeta.ru/politics/2021/09/03_a_13...,Из-за геополитической и экономической роли Рос...,Глава чешского МИДа призвал восстанавливать св...,После взрывов во Врбетице отношения между Чехи...
6791,2021-09-03 11:43:50,https://www.gazeta.ru/science/2021/09/03_a_139...,Международная группа астрономов впервые наблюд...,Астрономы наблюдали вспышку сверхновой нового ...,Астрономы впервые наблюдали процесс вспышки св...


In [15]:
df_m = df_test[:10]

In [22]:
df_m['summary_pred'] = df_m.apply(lambda x: summarize(x['text'], len(x['summary'].split())), axis=1)

<ipython-input-22-d263e440c7d9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_m['summary_pred'] = df_m.apply(lambda x: summarize(x['text'], len(x['summary'].split())), axis=1)


In [21]:
calc_lead_n_score(df_m, n=3)

TypeError: ignored

In [18]:
# AutoModelForSeq2SeqLM
calc_scores(df_m['summary'].values, df_m['summary_pred'].values, metric="all")

Count: 10
Ref: Революция «радикальных леваков» ждет Америку, если президентом США изберут демократа Джо Байдена, заявил Дональд Трамп. По словам действующего главы американского государства, его соперник на грядущих выборах — «марионетка». В то же время Байден разыгрывает карту «покорности» Трампа перед Россией.
Hyp: Президент США Дональд Трамп назвал «марионеткой» своего соперника, демократа Джо Байдена. В обзоре Би-би-си:
BLEU:  0.08310122193061632
ROUGE:  {'rouge-1': {'r': 0.12571633145690556, 'p': 0.2595312205910032, 'f': 0.16132017266919554}, 'rouge-2': {'r': 0.04646843884657599, 'p': 0.09271626521626522, 'f': 0.05917884396339366}, 'rouge-l': {'r': 0.10753165898651876, 'p': 0.2161817711274233, 'f': 0.13661043905351994}}


In [19]:
# T5ForConditionalGeneration
calc_scores(df_m['summary'].values, df_m['summary_pred'].values, metric="all")

Count: 10
Ref: Революция «радикальных леваков» ждет Америку, если президентом США изберут демократа Джо Байдена, заявил Дональд Трамп. По словам действующего главы американского государства, его соперник на грядущих выборах — «марионетка». В то же время Байден разыгрывает карту «покорности» Трампа перед Россией.
Hyp: Президент США Дональд Трамп назвал «марионеткой» своего соперника, демократа Джо Байдена. В обзоре Би-би-си:
BLEU:  0.08310122193061632
ROUGE:  {'rouge-1': {'r': 0.12571633145690556, 'p': 0.2595312205910032, 'f': 0.16132017266919554}, 'rouge-2': {'r': 0.04646843884657599, 'p': 0.09271626521626522, 'f': 0.05917884396339366}, 'rouge-l': {'r': 0.10753165898651876, 'p': 0.2161817711274233, 'f': 0.13661043905351994}}
